In [ ]:
from __future__ import print_function 

%load_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
%matplotlib inline

import os, sys
import numpy as np
import time

import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1*X GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=(1024*4))])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)



import pandas as pd
import pickle
import gc, re, copy
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils

# Project imports 
from data.tasks import *
from data.label_shift import label_shift_linear, plot_labeldist, plot_splitbars
from experiments.models import *
from experiments.training import *
from bounds.bounds import *
from util.batch import *
from util.kl import *
from util.misc import *

# Hyper-parameters
batch_size = 128
 

project_folder="/cephyr/users/adambre/Alvis/mnist_transfer/"
# project_folder="/cephyr/NOBACKUP/groups/snic2021-23-538/mnist_transfer/"

# Load and structure data

In [ ]:
task = 4
x_source,y_source,x_target,y_target=load_task(task)

# Run evaluation 

In [ ]:

architecture="lenet"
delta=0.05
binary=True
epsilons=[0.01]
alphas=[0]#0,0.3]
sigmas=[[3,2]]
seed=69105
for alpha in alphas:
    print("alpha:"+str(alpha))
    if alpha==0:
        x_bound=x_source
        y_bound=y_source
    else:
        x_bound, x_prior, y_bound , y_prior = train_test_split(x_source,y_source,test_size=alpha,random_state=seed)
    for epsilon in epsilons:
        print("epsilon:"+str(epsilon))
        for sigma in sigmas:    
            print("sigma:"+str(sigma))
            paths=posterior_checkpoints(task, epsilon, alpha, binary=binary,architecture="lenet")
            for p in paths:
                results=compute_bound_parts(task,p,x_bound,y_bound,x_target,y_target,alpha=alpha,sigma=sigma,epsilon=epsilon,binary=binary,architecture=architecture)
                
                checkpoint = results['checkpoint'].values.ravel()[0]

                if binary:
                    result_path=project_folder+"results/"+"task"+str(task)+"/Binary/"+str(architecture)+"/"+str(int(1000*epsilon))+\
                        "_"+str(int(100*alpha))+"_"+str(sigma[0])+str(sigma[1])+'_'+str(seed)+'_'+checkpoint+'_results.pkl'
                else:
                    result_path=project_folder+"results/"+"task"+str(task)+"/"+str(architecture)+"/"+str(int(1000*epsilon))+"_"+str(int(100*alpha))+\
                    "_"+str(sigma[0])+str(sigma[1])+'_'+str(seed)+'_'+checkpoint+'_results.pkl'

                # Create dir
                os.makedirs(os.path.dirname(result_path), exist_ok=True)
                results.to_pickle(result_path)